In [ ]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]
!pip install plotly


     |████████████████████████████████| 4.7 MB 6.7 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
     |████████████████████████████████| 596 kB 72.9 MB/s 
     |████████████████████████████████| 316 kB 9.9 MB/s 
     |████████████████████████████████| 1.4 MB 62.8 MB/s 
     |████████████████████████████████| 233 kB 70.9 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 175 kB 14.1 MB/s 
     |████████████████████████████████| 235 kB 63.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


import matplotlib.pyplot as plt
import hazm
from hazm import Lemmatizer
from cleantext import clean
import nltk


import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
data = pd.read_csv('dataset1_org.csv',header=None)
data.drop(0,inplace=True,axis=1)
data

,1,2
0,پرسش,@pejvak313 ارتباطش رو با واکسن توضیح میدین؟
1,پرسش,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...
2,پرسش,@ly69204411 سازمان بهداشت جهاني خودش امار ميده...
3,پرسش,@kimnacri واسه کرونا؟
4,پرسش,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...
...,...,...
7263,پرسش,به‌نظر شما عامل اصلی کمبود واردات واکسن #کرونا...
7264,خنثی,@amoyepesarshoja @bahar_sh_y منی که عکس با ماس...
7265,عصبانیت,به نظرم لیست کشته شده های این مدت در اثر آلودگ...
7266,غم,@kimiyaws 🤦🏻‍♀️ کرونا


In [ ]:
data[1].replace('ترس','ترس ',inplace=True)

In [ ]:
data[1].unique()

array(['پرسش', 'ترس ', 'تعجب', 'خنثی', 'خنده', 'خوشحالی', 'عصبانیت', 'غم'],
      dtype=object)

In [ ]:
with open('stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
nltk_stopwords = [str(line).replace('\n', '') for line in stopwords]

In [ ]:
data['comment_len_by_words'] = data[2].apply(lambda t: len(hazm.word_tokenize(t)))
data


,1,2,comment_len_by_words
0,پرسش,@pejvak313 ارتباطش رو با واکسن توضیح میدین؟,9
1,پرسش,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...,57
2,پرسش,@ly69204411 سازمان بهداشت جهاني خودش امار ميده...,31
3,پرسش,@kimnacri واسه کرونا؟,4
4,پرسش,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...,65
...,...,...,...
7263,پرسش,به‌نظر شما عامل اصلی کمبود واردات واکسن #کرونا...,15
7264,خنثی,@amoyepesarshoja @bahar_sh_y منی که عکس با ماس...,9
7265,عصبانیت,به نظرم لیست کشته شده های این مدت در اثر آلودگ...,27
7266,غم,@kimiyaws 🤦🏻‍♀️ کرونا,3


In [ ]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 75


In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
data_gl_than(data, 256, 3)


Texts with word length of greater than 3 and less than 256 includes 99.53% of the whole!


In [ ]:
# remove comments with the length of fewer than three words
minlim, maxlim = 3, 256

data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
data

,1,2,comment_len_by_words
0,پرسش,@pejvak313 ارتباطش رو با واکسن توضیح میدین؟,9.0
1,پرسش,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...,57.0
2,پرسش,@ly69204411 سازمان بهداشت جهاني خودش امار ميده...,31.0
3,پرسش,@kimnacri واسه کرونا؟,4.0
4,پرسش,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...,65.0
...,...,...,...
7229,غم,@MahdiarSaeedian این نمونه که خدمتتون عرض کردم...,64.0
7230,پرسش,به‌نظر شما عامل اصلی کمبود واردات واکسن #کرونا...,15.0
7231,خنثی,@amoyepesarshoja @bahar_sh_y منی که عکس با ماس...,9.0
7232,عصبانیت,به نظرم لیست کشته شده های این مدت در اثر آلودگ...,27.0


In [ ]:
data['cleaned_comment'] = data[2].apply(cleaning)

# calculate the length of comments based on their words
data['cleaned_comment_len_by_words'] = data['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
data = data.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)

data.head()

,1,2,comment_len_by_words,cleaned_comment,cleaned_comment_len_by_words
0,پرسش,@pejvak313 ارتباطش رو با واکسن توضیح میدین؟,9.0,@pejvak۳۱۳ ارتباطش رو با واکسن توضیح میدین؟,9
1,پرسش,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...,57.0,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...,56
2,پرسش,@ly69204411 سازمان بهداشت جهاني خودش امار ميده...,31.0,@ly۶۹۲۰۴۴۱۱ سازمان بهداشت جهانی خودش امار میده...,30
3,پرسش,@kimnacri واسه کرونا؟,4.0,@kimnacri واسه کرونا؟,4
4,پرسش,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...,65.0,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...,65


In [ ]:
data['tweet_without_stopwords'] = data['cleaned_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))


In [ ]:
data = data[[1,'cleaned_comment']]
data.columns = ['label','comment']
data.head()

,label,comment
0,پرسش,@pejvak۳۱۳ ارتباطش رو با واکسن توضیح میدین؟
1,پرسش,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...
2,پرسش,@ly۶۹۲۰۴۴۱۱ سازمان بهداشت جهانی خودش امار میده...
3,پرسش,@kimnacri واسه کرونا؟
4,پرسش,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...


In [ ]:
fig = go.Figure()

groupby_label = data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
labels = list(sorted(data['label'].unique()))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}
print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'ترس ': 0, 'تعجب': 1, 'خنثی': 2, 'خنده': 3, 'خوشحالی': 4, 'عصبانیت': 5, 'غم': 6, 'پرسش': 7}
id2label: {0: 'ترس ', 1: 'تعجب', 2: 'خنثی', 3: 'خنده', 4: 'خوشحالی', 5: 'عصبانیت', 6: 'غم', 7: 'پرسش'}


In [ ]:

data['label_id'] = data['label'].apply(lambda t: labels.index(t))

train, test = train_test_split(data, test_size=0.1, random_state=1, stratify=data['label'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['comment'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['comment'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)


(5859, 3)
(651, 3)
(724, 3)


In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u062a\u0631\u0633 ",
    "1": "\u062a\u0639\u062c\u0628",
    "2": "\u062e\u0646\u062b\u06cc",
    "3": "\u062e\u0646\u062f\u0647",
    "4": "\u062e\u0648\u0634\u062d\u0627\u0644\u06cc",
    "5": "\u0639\u0635\u0628\u0627\u0646\u06cc\u062a",
    "6": "\u063a\u0645",
    "7": "\u067e\u0631\u0633\u0634"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u062a\u0631\u0633 ": 0,
    "\u062a\u0639\u062c\u0628": 1,
    "\u062e\u0646\u062b\u06cc": 2,
    "\u062e\u0646\u062f\u0647": 3,
    "\u062e\u0648\u0634\u062d\u0627\u0644\u06cc": 4,
    "\u0639\u0635\u0628\u0627\u0646\u06cc\u062a": 5,
    "\u063a\u0645": 6,
    "\u067e\u0631\u0633\u0634": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_a

In [ ]:
data

,label,comment,label_id
0,پرسش,@pejvak۳۱۳ ارتباطش رو با واکسن توضیح میدین؟,7
1,پرسش,@mamadporii نه میدونی فشار اومده بدجور.۲سال کر...,7
2,پرسش,@ly۶۹۲۰۴۴۱۱ سازمان بهداشت جهانی خودش امار میده...,7
3,پرسش,@kimnacri واسه کرونا؟,7
4,پرسش,@raziehtaheryan کاهش شیب تورم، یه امر طبیعی بو...,7
...,...,...,...
7229,غم,@mahdiarsaeedian این نمونه که خدمتتون عرض کردم...,6
7230,پرسش,به‌نظر شما عامل اصلی کمبود واردات واکسن کرونا ...,7
7231,خنثی,@amoyepesarshoja @bahar_sh_y منی که عکس با ماس...,2
7232,عصبانیت,به نظرم لیست کشته شده‌های این مدت در اثر آلودگ...,5


In [ ]:
idx = np.random.randint(0, len(train))
sample_comment = train.iloc[idx]['comment']
sample_label = train.iloc[idx]['label']

print(f'Sample: \n{sample_comment}\n{sample_label}')


Sample: 
بچای علوم پزشکی کرونا نمیگیرن، فقط ما فرهنگیانیا کرونا میگیریم! من دانشگاه میخواممممممم.
غم


In [ ]:
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_comment}')
print(f'Token IDs: {token_ids}')

  Comment: بچای علوم پزشکی کرونا نمیگیرن، فقط ما فرهنگیانیا کرونا میگیریم! من دانشگاه میخواممممممم.
Token IDs: [11916, 2784, 4458, 4640, 12646, 29077, 2011, 1348, 3765, 2964, 13456, 2874, 12646, 12109, 1001, 2842, 3363, 17953, 14469, 14469, 14469, 1012]


In [ ]:
encoding = tokenizer.encode_plus(
    sample_comment,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2, 11916,  2784,  4458,  4640, 12646, 29077,  2011,  1348,  3765,
          2964, 13456,  2874, 12646, 12109,  1001,  2842,  3363, 17953, 14469,
         14469, 14469,  1012,     4,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    
    features = glue_convert_examples_to_features(
        examples, 
        tokenizer, 
        maxlen, 
        output_mode=output_mode, 
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [ ]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=128)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_valid, y_valid, maxlen=128)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128, is_tf_dataset=False)

  0%|          | 0/5859 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:66: FutureWarning:

This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py



  0%|          | 0/5859 [00:00<?, ?it/s]

  0%|          | 0/651 [00:00<?, ?it/s]

  0%|          | 0/651 [00:00<?, ?it/s]

  0%|          | 0/724 [00:00<?, ?it/s]

  0%|          | 0/724 [00:00<?, ?it/s]

  0%|          | 0/724 [00:00<?, ?it/s]

  0%|          | 0/724 [00:00<?, ?it/s]

In [ ]:
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'        target: {value[1]}')

     input_ids: [    2  1020 83314  2030     1  1455 24961 51088 25725  2077  2050  1348
  3245  1379  3372 13558  1350 48449  2831  3163  1039  2822 12646  3296
  1350     4     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
        target: 7


In [ ]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)

    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
valid_dataset = get_training_dataset(valid_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
valid_steps = len(valid_examples) // VALID_BATCH_SIZE

train_steps, valid_steps

(366, 40)

In [ ]:
def build_model(model_name, config, learning_rate=3e-5):
    model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [ ]:
model = build_model(MODEL_NAME_OR_PATH, config, learning_rate=LEARNING_RATE)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%%time

r = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=EPOCHS,
    verbose=1)

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))

Epoch 1/3
366/366 [==============================] - 200s 468ms/step - loss: 1.7883 - accuracy: 0.3578 - val_loss: 1.6226 - val_accuracy: 0.4703
Epoch 2/3
366/366 [==============================] - 170s 464ms/step - loss: 1.1843 - accuracy: 0.6224 - val_loss: 1.0891 - val_accuracy: 0.6719
Epoch 3/3
366/366 [==============================] - 170s 463ms/step - loss: 0.5016 - accuracy: 0.8600 - val_loss: 0.7347 - val_accuracy: 0.7937
FINAL ACCURACY MEAN:  0.6453124980131785
CPU times: user 6min 8s, sys: 58.4 s, total: 7min 6s
Wall time: 9min 49s


In [ ]:
model.save_pretrained(os.path.dirname(OUTPUT_PATH))


In [ ]:
ev = model.evaluate(test_dataset_base.batch(TEST_BATCH_SIZE))
print()
print(f'Evaluation: {ev}')
print()

predictions = model.predict(xtest)
ypred = predictions[0].argmax(axis=-1).tolist()

print()
print(classification_report(ytest, ypred, target_names=labels))
print()

print(f'F1: {f1_score(ytest, ypred, average="weighted")}')

46/46 [==============================] - 8s 161ms/step - loss: 0.8176 - accuracy: 0.7956

Evaluation: [0.8175584077835083, 0.7955800890922546]


              precision    recall  f1-score   support

        ترس        0.80      0.63      0.71        57
        تعجب       0.77      0.74      0.75        68
        خنثی       0.82      0.79      0.80       179
        خنده       0.94      0.70      0.80        63
     خوشحالی       0.72      0.72      0.72        60
     عصبانیت       0.70      0.95      0.81        98
          غم       0.82      0.84      0.83       118
        پرسش       0.85      0.86      0.86        81

    accuracy                           0.80       724
   macro avg       0.80      0.78      0.78       724
weighted avg       0.80      0.80      0.79       724


F1: 0.7944161823820468
